In [ ]:
class ReadWrite:
  def read_csv_remove_row(path):
    df = spark.read.csv(path, header=True, inferSchema=True, quote='"', escape='"', multiLine=True)
    return df.withColumn("id", monotonically_increasing_id()).filter(col('id') > 1 ).drop('id')

  def schema_apply_change_col(df, df_schema, clm_rename_dic):
    df = spark.createDataFrame(df.rdd, df_schema)
    return reduce(lambda temp_df, old_col : temp_df.withColumnRenamed(old_col, clm_rename_dic[old_col]), clm_rename_dic, df)

  def result_dfs(merge_col, result_df):
    is_null_exp = " & ".join(f'(col("{col}").isNull())' for col in merge_col)
    is_not_null_exp = " & ".join(f'(col("{col}").isNotNull())' for col in merge_col)
    return result_df.filter(eval(is_null_exp)), result_df.filter(eval(is_not_null_exp))

  def add_load_date(df, schedule_date):
    return df.withColumn('load_date', to_timestamp(lit(schedule_date)))

  def get_schema_rename_clm(table_name):
    if table_name == 'onboarding_frigidaire' :
      return onboarding_frigidaire_schema, onboarding_frigidaire_clm_rename
    else:
      raise('Given Table name not found')